In [6]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import log_loss, roc_auc_score
from deepctr.models import DeepFM,PNN
from deepctr.models import FGCNN
from deepctr.inputs import  SparseFeat, DenseFeat,get_feature_names
import math
import numpy as np
from keras.callbacks import EarlyStopping
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters


Using TensorFlow backend.


In [21]:
import numpy as np
data = pd.read_csv('./dac/avazu/avazu_sampled.csv')

data=data.astype({'hour': 'int32'})
dense_features = ["C1"]+['C'+str(i) for i in range(14, 22)]
sparse_features = ["hour","banner_pos","site_id","site_domain","site_category","app_id","app_domain","app_category","device_id","device_ip","device_model","device_type","device_conn_type"]

# 
data[sparse_features] = data[sparse_features].fillna('-1', )
data[dense_features] = data[dense_features].fillna(0,)
target = 'click'
# 1.Label Encoding for sparse features,and do simple Transformation for dense features
for feat in sparse_features:
    lbe = LabelEncoder()
#     data[feat]=data[feat].apply(lambda x: str(x))
    data[feat] = lbe.fit_transform(data[feat])
mms = MinMaxScaler(feature_range=(0, 1))
data[dense_features] = mms.fit_transform(data[dense_features])

# 2.count #unique features for each sparse field,and record dense feature field name

fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=data[feat].nunique(),embedding_dim=40)
                       for i,feat in enumerate(sparse_features)] + [DenseFeat(feat, 1,) for feat in dense_features]
print(np.array(fixlen_feature_columns).shape)
dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

# 3.generate input data for model

train, test = train_test_split(data, test_size=0.2)

train_model_input = {name:train[name] for name in feature_names}
test_model_input = {name:test[name] for name in feature_names}


(22,)


In [34]:
import tensorflow as tf

from deepctr.inputs import input_from_feature_columns, build_input_features, combined_dnn_input
from deepctr.layers.core import PredictionLayer, DNN
from deepctr.layers.interaction import InnerProductLayer, OutterProductLayer
from deepctr.layers.utils import concat_func
class NoMask(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(NoMask, self).__init__(**kwargs)

    def build(self, input_shape):
        # Be sure to call this somewhere!
        super(NoMask, self).build(input_shape)

    def call(self, x, mask=None, **kwargs):
        return x

    def compute_mask(self, inputs, mask):
        return None
    
def concat_func(inputs, axis=-1, mask=False):
    print("start concat")
    if not mask:
        inputs = list(map(NoMask(), inputs))
    if len(inputs) == 1:
        return inputs[0]
    else:
        print("Calling keras Concatenate")
        print(inputs)
        return tf.keras.layers.Concatenate(axis=axis)(inputs)

def PNN(dnn_feature_columns, embedding_size=8, dnn_hidden_units=(128, 128), l2_reg_embedding=1e-5, l2_reg_dnn=0,
        init_std=0.0001, seed=1024, dnn_dropout=0, dnn_activation='relu', use_inner=True, use_outter=False,
        kernel_type='mat', task='binary'):
    """Instantiates the Product-based Neural Network architecture.
    :param dnn_feature_columns: An iterable containing all the features used by deep part of the model.
    :param embedding_size: positive integer,sparse feature embedding_size
    :param dnn_hidden_units: list,list of positive integer or empty list, the layer number and units in each layer of deep net
    :param l2_reg_embedding: float . L2 regularizer strength applied to embedding vector
    :param l2_reg_dnn: float. L2 regularizer strength applied to DNN
    :param init_std: float,to use as the initialize std of embedding vector
    :param seed: integer ,to use as random seed.
    :param dnn_dropout: float in [0,1), the probability we will drop out a given DNN coordinate.
    :param dnn_activation: Activation function to use in DNN
    :param use_inner: bool,whether use inner-product or not.
    :param use_outter: bool,whether use outter-product or not.
    :param kernel_type: str,kernel_type used in outter-product,can be ``'mat'`` , ``'vec'`` or ``'num'``
    :param task: str, ``"binary"`` for  binary logloss or  ``"regression"`` for regression loss
    :return: A Keras model instance.
    """
    print("PNN start")
    if kernel_type not in ['mat', 'vec', 'num']:
        raise ValueError("kernel_type must be mat,vec or num")

    features = build_input_features(dnn_feature_columns)

    inputs_list = list(features.values())

    sparse_embedding_list, dense_value_list = input_from_feature_columns(features, dnn_feature_columns,
                                                                         l2_reg_embedding, init_std, seed)
    print("inner product start")
    print(sparse_embedding_list)
    inner_product = tf.keras.layers.Flatten()(InnerProductLayer()(sparse_embedding_list))
    print("inner product end")
    outter_product = OutterProductLayer(kernel_type)(sparse_embedding_list)
    
    
    print("outer")
    # ipnn deep input
    linear_signal = tf.keras.layers.Reshape(
        [len(sparse_embedding_list) * embedding_size])(concat_func(sparse_embedding_list))
    

    if use_inner and use_outter:
        deep_input = tf.keras.layers.Concatenate()(
            [linear_signal, inner_product, outter_product])
    elif use_inner:
        deep_input = tf.keras.layers.Concatenate()(
            [linear_signal, inner_product])
    elif use_outter:
        deep_input = tf.keras.layers.Concatenate()(
            [linear_signal, outter_product])
    else:
        deep_input = linear_signal

    dnn_input = combined_dnn_input([deep_input], dense_value_list)
    dnn_out = DNN(dnn_hidden_units, dnn_activation, l2_reg_dnn, dnn_dropout,
                  False, seed)(dnn_input)
    dnn_logit = tf.keras.layers.Dense(
        1, use_bias=False, activation=None)(dnn_out)

    output = PredictionLayer(task)(dnn_logit)

    model = tf.keras.models.Model(inputs=inputs_list,outputs=output)
    return model

In [38]:
from tensorflow.python.keras.layers import Layer
class InnerProductLayer(Layer):
    """InnerProduct Layer used in PNN that compute the element-wise
    product or inner product between feature vectors.
      Input shape
        - a list of 3D tensor with shape: ``(batch_size,1,embedding_size)``.
      Output shape
        - 3D tensor with shape: ``(batch_size, N*(N-1)/2 ,1)`` if use reduce_sum. or 3D tensor with shape: ``(batch_size, N*(N-1)/2, embedding_size )`` if not use reduce_sum.
      Arguments
        - **reduce_sum**: bool. Whether return inner product or element-wise product
      References
            - [Qu Y, Cai H, Ren K, et al. Product-based neural networks for user response prediction[C]//Data Mining (ICDM), 2016 IEEE 16th International Conference on. IEEE, 2016: 1149-1154.](https://arxiv.org/pdf/1611.00144.pdf)
    """

    def __init__(self, reduce_sum=True, **kwargs):
        self.reduce_sum = reduce_sum
        super(InnerProductLayer, self).__init__(**kwargs)

    def build(self, input_shape):

        if not isinstance(input_shape, list) or len(input_shape) < 2:
            raise ValueError('A `InnerProductLayer` layer should be called '
                             'on a list of at least 2 inputs')

        reduced_inputs_shapes = [shape.as_list() for shape in input_shape]
        shape_set = set()

        for i in range(len(input_shape)):
            shape_set.add(tuple(reduced_inputs_shapes[i]))

        if len(shape_set) > 1:
            raise ValueError('A `InnerProductLayer` layer requires '
                             'inputs with same shapes '
                             'Got different shapes: %s' % (shape_set))

        if len(input_shape[0]) != 3 or input_shape[0][1] != 1:
            raise ValueError('A `InnerProductLayer` layer requires '
                             'inputs of a list with same shape tensor like (None,1,embedding_size)'
                             'Got different shapes: %s' % (input_shape[0]))
        super(InnerProductLayer, self).build(
            input_shape)  # Be sure to call this somewhere!

    def call(self, inputs, **kwargs):
        if K.ndim(inputs[0]) != 3:
            raise ValueError(
                "Unexpected inputs dimensions %d, expect to be 3 dimensions" % (K.ndim(inputs)))

        embed_list = inputs
        row = []
        col = []
        num_inputs = len(embed_list)

        for i in range(num_inputs - 1):
            for j in range(i + 1, num_inputs):
                row.append(i)
                col.append(j)
        p = tf.concat([embed_list[idx]
                       for idx in row], axis=1)  # batch num_pairs k
        q = tf.concat([embed_list[idx]
                       for idx in col], axis=1)

        inner_product = p * q
        if self.reduce_sum:
            inner_product = reduce_sum(
                inner_product, axis=2, keep_dims=True)
        return inner_product

    def compute_output_shape(self, input_shape):
        num_inputs = len(input_shape)
        num_pairs = int(num_inputs * (num_inputs - 1) / 2)
        input_shape = input_shape[0]
        embed_size = input_shape[-1]
        if self.reduce_sum:
            return (input_shape[0], num_pairs, 1)
        else:
            return (input_shape[0], num_pairs, embed_size)

    def get_config(self, ):
        config = {'reduce_sum': self.reduce_sum, }
        base_config = super(InnerProductLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [35]:
# 4.Define Model,train,predict and evaluate
model = PNN(linear_feature_columns, dnn_feature_columns, task='binary')
model.compile("adam", "binary_crossentropy",metrics=['binary_crossentropy'])
earlystopping_callback = EarlyStopping(monitor='val_binary_crossentropy',verbose=1,patience=1,baseline=None)
history = model.fit(train_model_input, train[target].values,
                    batch_size=1024, epochs=15, verbose=2, validation_split=0.2)
pred_ans = model.predict(test_model_input, batch_size=256)

PNN start
inner product start
[<tf.Tensor 'sparse_emb_hour_12/Identity:0' shape=(None, 1, 40) dtype=float32>, <tf.Tensor 'sparse_emb_banner_pos_12/Identity:0' shape=(None, 1, 40) dtype=float32>, <tf.Tensor 'sparse_emb_site_id_12/Identity:0' shape=(None, 1, 40) dtype=float32>, <tf.Tensor 'sparse_emb_site_domain_12/Identity:0' shape=(None, 1, 40) dtype=float32>, <tf.Tensor 'sparse_emb_site_category_12/Identity:0' shape=(None, 1, 40) dtype=float32>, <tf.Tensor 'sparse_emb_app_id_12/Identity:0' shape=(None, 1, 40) dtype=float32>, <tf.Tensor 'sparse_emb_app_domain_12/Identity:0' shape=(None, 1, 40) dtype=float32>, <tf.Tensor 'sparse_emb_app_category_12/Identity:0' shape=(None, 1, 40) dtype=float32>, <tf.Tensor 'sparse_emb_device_id_12/Identity:0' shape=(None, 1, 40) dtype=float32>, <tf.Tensor 'sparse_emb_device_ip_12/Identity:0' shape=(None, 1, 40) dtype=float32>, <tf.Tensor 'sparse_emb_device_model_12/Identity:0' shape=(None, 1, 40) dtype=float32>, <tf.Tensor 'sparse_emb_device_type_12/Ide

TypeError: Expected int32, got 'hour' of type 'str' instead.

In [12]:
import kerastuner
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow import keras
def buildFGCNN(hp):
    # k=40
    # conv=7*1
    #Kernal=[14,16,18,20]
    #new=[3,3,3,3]
    # ruenet=[4096,2048,1/]
    activation = hp.Choice('activation', ['relu', 'selu','elu'])
    dropout_rate = hp.Float(
                'dropout_rate', 0.0, 0.8, step=0.1, default=0.5)
    model = FGCNN(linear_feature_columns, dnn_feature_columns,dnn_dropout=0.8,conv_kernel_width=(7, 7, 7), conv_filters=(14, 16, 18),
          new_maps=(3, 3, 3),
          pooling_width=( 2, 2,2),  task='binary')
    model.compile( optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4,1e-4])), loss="binary_crossentropy",metrics=['binary_crossentropy'])
    return model


class MyTuner(kerastuner.tuners.BayesianOptimization):
    def run_trial(self, trial, *args, **kwargs):
        # You can add additional HyperParameters for preprocessing and custom training loops
        # via overriding `run_trial`
        kwargs['batch_size'] = trial.hyperparameters.Int('batch_size', 256, 1024, step=256)
        kwargs['epochs'] = trial.hyperparameters.Int('epochs', 10, 30, step=5)
        super(MyTuner, self).run_trial(trial, *args, **kwargs)
  
def HypermeterTune(model,directory):
    tuner = MyTuner(
        buildFGCNN,
        objective='val_binary_crossentropy',
        max_trials=5,
        executions_per_trial=2,
        directory=directory)

    tuner.search_space_summary()

    tuner.search(x=train_model_input,
                 y=train[target].values,callbacks=[tf.keras.callbacks.EarlyStopping('val_binary_crossentropy', patience=1)],
                 validation_data=(test_model_input,test[target].values))

    return tuner


In [ ]:
FNNtuner=HypermeterTune("buildFGCNN","turnFGCNN1")

INFO:tensorflow:Reloading Oracle from turnFGCNN1/untitled_project/oracle.json
FGCNN start
origin_input shape: (None, 13, 40)
INFO:tensorflow:Reloading Tuner from turnFGCNN1/untitled_project/tuner0.json


FGCNN start
origin_input shape: (None, 13, 40)
Train on 418073 samples, validate on 104519 samples
Epoch 1/15


/home/yxy/anaconda3/envs/py37/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/yxy/anaconda3/envs/py37/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


 74752/418073 [====>.........................] - ETA: 59:56 - loss: 0.7296 - binary_crossentropy: 0.72 - ETA: 31:10 - loss: 0.7215 - binary_crossentropy: 0.72 - ETA: 21:32 - loss: 0.7099 - binary_crossentropy: 0.70 - ETA: 16:47 - loss: 0.7021 - binary_crossentropy: 0.70 - ETA: 13:53 - loss: 0.6923 - binary_crossentropy: 0.69 - ETA: 11:57 - loss: 0.6843 - binary_crossentropy: 0.68 - ETA: 10:33 - loss: 0.6751 - binary_crossentropy: 0.67 - ETA: 9:30 - loss: 0.6701 - binary_crossentropy: 0.6701 - ETA: 8:41 - loss: 0.6654 - binary_crossentropy: 0.665 - ETA: 8:02 - loss: 0.6557 - binary_crossentropy: 0.655 - ETA: 7:30 - loss: 0.6475 - binary_crossentropy: 0.647 - ETA: 7:03 - loss: 0.6419 - binary_crossentropy: 0.641 - ETA: 6:41 - loss: 0.6368 - binary_crossentropy: 0.636 - ETA: 6:22 - loss: 0.6292 - binary_crossentropy: 0.629 - ETA: 6:04 - loss: 0.6240 - binary_crossentropy: 0.623 - ETA: 5:50 - loss: 0.6168 - binary_crossentropy: 0.616 - ETA: 5:36 - loss: 0.6104 - binary_crossentropy: 0.610 

149504/418073 [=========>....................] - ETA: 2:09 - loss: 0.4592 - binary_crossentropy: 0.459 - ETA: 2:09 - loss: 0.4590 - binary_crossentropy: 0.459 - ETA: 2:09 - loss: 0.4588 - binary_crossentropy: 0.458 - ETA: 2:08 - loss: 0.4587 - binary_crossentropy: 0.458 - ETA: 2:08 - loss: 0.4581 - binary_crossentropy: 0.458 - ETA: 2:08 - loss: 0.4576 - binary_crossentropy: 0.457 - ETA: 2:07 - loss: 0.4572 - binary_crossentropy: 0.457 - ETA: 2:07 - loss: 0.4570 - binary_crossentropy: 0.457 - ETA: 2:07 - loss: 0.4568 - binary_crossentropy: 0.456 - ETA: 2:06 - loss: 0.4567 - binary_crossentropy: 0.456 - ETA: 2:06 - loss: 0.4563 - binary_crossentropy: 0.456 - ETA: 2:06 - loss: 0.4562 - binary_crossentropy: 0.456 - ETA: 2:06 - loss: 0.4560 - binary_crossentropy: 0.456 - ETA: 2:05 - loss: 0.4558 - binary_crossentropy: 0.455 - ETA: 2:05 - loss: 0.4557 - binary_crossentropy: 0.455 - ETA: 2:05 - loss: 0.4553 - binary_crossentropy: 0.455 - ETA: 2:04 - loss: 0.4551 - binary_crossentropy: 0.455 -

...

 74752/418073 [====>.........................] - ETA: 2:16 - loss: 0.3199 - binary_crossentropy: 0.318 - ETA: 2:16 - loss: 0.3437 - binary_crossentropy: 0.342 - ETA: 2:14 - loss: 0.3524 - binary_crossentropy: 0.351 - ETA: 2:13 - loss: 0.3584 - binary_crossentropy: 0.357 - ETA: 2:12 - loss: 0.3547 - binary_crossentropy: 0.353 - ETA: 2:14 - loss: 0.3474 - binary_crossentropy: 0.346 - ETA: 2:13 - loss: 0.3435 - binary_crossentropy: 0.342 - ETA: 2:13 - loss: 0.3444 - binary_crossentropy: 0.343 - ETA: 2:12 - loss: 0.3449 - binary_crossentropy: 0.343 - ETA: 2:12 - loss: 0.3448 - binary_crossentropy: 0.343 - ETA: 2:12 - loss: 0.3462 - binary_crossentropy: 0.345 - ETA: 2:12 - loss: 0.3431 - binary_crossentropy: 0.341 - ETA: 2:12 - loss: 0.3396 - binary_crossentropy: 0.338 - ETA: 2:12 - loss: 0.3370 - binary_crossentropy: 0.335 - ETA: 2:12 - loss: 0.3343 - binary_crossentropy: 0.333 - ETA: 2:12 - loss: 0.3338 - binary_crossentropy: 0.332 - ETA: 2:12 - loss: 0.3314 - binary_crossentropy: 0.330 -

224256/418073 [===============>..............] - ETA: 1:23 - loss: 0.3158 - binary_crossentropy: 0.314 - ETA: 1:23 - loss: 0.3160 - binary_crossentropy: 0.314 - ETA: 1:23 - loss: 0.3160 - binary_crossentropy: 0.314 - ETA: 1:23 - loss: 0.3160 - binary_crossentropy: 0.314 - ETA: 1:23 - loss: 0.3161 - binary_crossentropy: 0.314 - ETA: 1:22 - loss: 0.3161 - binary_crossentropy: 0.314 - ETA: 1:22 - loss: 0.3162 - binary_crossentropy: 0.314 - ETA: 1:22 - loss: 0.3162 - binary_crossentropy: 0.314 - ETA: 1:22 - loss: 0.3162 - binary_crossentropy: 0.314 - ETA: 1:22 - loss: 0.3163 - binary_crossentropy: 0.314 - ETA: 1:22 - loss: 0.3164 - binary_crossentropy: 0.314 - ETA: 1:21 - loss: 0.3164 - binary_crossentropy: 0.314 - ETA: 1:21 - loss: 0.3165 - binary_crossentropy: 0.315 - ETA: 1:21 - loss: 0.3165 - binary_crossentropy: 0.315 - ETA: 1:21 - loss: 0.3166 - binary_crossentropy: 0.315 - ETA: 1:21 - loss: 0.3167 - binary_crossentropy: 0.315 - ETA: 1:21 - loss: 0.3167 - binary_crossentropy: 0.315 -

301568/418073 [====================>.........] - ETA: 1:00 - loss: 0.3170 - binary_crossentropy: 0.315 - ETA: 1:00 - loss: 0.3171 - binary_crossentropy: 0.315 - ETA: 1:00 - loss: 0.3170 - binary_crossentropy: 0.315 - ETA: 1:00 - loss: 0.3171 - binary_crossentropy: 0.315 - ETA: 59s - loss: 0.3172 - binary_crossentropy: 0.315 - ETA: 59s - loss: 0.3172 - binary_crossentropy: 0.31 - ETA: 59s - loss: 0.3171 - binary_crossentropy: 0.31 - ETA: 59s - loss: 0.3170 - binary_crossentropy: 0.31 - ETA: 59s - loss: 0.3171 - binary_crossentropy: 0.31 - ETA: 59s - loss: 0.3171 - binary_crossentropy: 0.31 - ETA: 58s - loss: 0.3171 - binary_crossentropy: 0.31 - ETA: 58s - loss: 0.3172 - binary_crossentropy: 0.31 - ETA: 58s - loss: 0.3171 - binary_crossentropy: 0.31 - ETA: 58s - loss: 0.3172 - binary_crossentropy: 0.31 - ETA: 58s - loss: 0.3173 - binary_crossentropy: 0.31 - ETA: 58s - loss: 0.3172 - binary_crossentropy: 0.31 - ETA: 57s - loss: 0.3173 - binary_crossentropy: 0.31 - ETA: 57s - loss: 0.3173 

378880/418073 [==========================>...] - ETA: 37s - loss: 0.3206 - binary_crossentropy: 0.31 - ETA: 37s - loss: 0.3207 - binary_crossentropy: 0.31 - ETA: 37s - loss: 0.3207 - binary_crossentropy: 0.31 - ETA: 36s - loss: 0.3207 - binary_crossentropy: 0.31 - ETA: 36s - loss: 0.3206 - binary_crossentropy: 0.31 - ETA: 36s - loss: 0.3206 - binary_crossentropy: 0.31 - ETA: 36s - loss: 0.3206 - binary_crossentropy: 0.31 - ETA: 36s - loss: 0.3206 - binary_crossentropy: 0.31 - ETA: 36s - loss: 0.3206 - binary_crossentropy: 0.31 - ETA: 36s - loss: 0.3206 - binary_crossentropy: 0.31 - ETA: 35s - loss: 0.3206 - binary_crossentropy: 0.31 - ETA: 35s - loss: 0.3206 - binary_crossentropy: 0.31 - ETA: 35s - loss: 0.3206 - binary_crossentropy: 0.31 - ETA: 35s - loss: 0.3207 - binary_crossentropy: 0.31 - ETA: 35s - loss: 0.3207 - binary_crossentropy: 0.31 - ETA: 35s - loss: 0.3207 - binary_crossentropy: 0.31 - ETA: 35s - loss: 0.3208 - binary_crossentropy: 0.31 - ETA: 34s - loss: 0.3208 - binary_

417792/418073 [============================>.] - ETA: 13s - loss: 0.3213 - binary_crossentropy: 0.31 - ETA: 12s - loss: 0.3213 - binary_crossentropy: 0.31 - ETA: 12s - loss: 0.3214 - binary_crossentropy: 0.31 - ETA: 12s - loss: 0.3215 - binary_crossentropy: 0.31 - ETA: 12s - loss: 0.3216 - binary_crossentropy: 0.31 - ETA: 12s - loss: 0.3216 - binary_crossentropy: 0.31 - ETA: 12s - loss: 0.3215 - binary_crossentropy: 0.31 - ETA: 11s - loss: 0.3215 - binary_crossentropy: 0.31 - ETA: 11s - loss: 0.3216 - binary_crossentropy: 0.31 - ETA: 11s - loss: 0.3216 - binary_crossentropy: 0.31 - ETA: 11s - loss: 0.3216 - binary_crossentropy: 0.31 - ETA: 11s - loss: 0.3215 - binary_crossentropy: 0.31 - ETA: 11s - loss: 0.3216 - binary_crossentropy: 0.31 - ETA: 10s - loss: 0.3215 - binary_crossentropy: 0.31 - ETA: 10s - loss: 0.3215 - binary_crossentropy: 0.31 - ETA: 10s - loss: 0.3216 - binary_crossentropy: 0.31 - ETA: 10s - loss: 0.3216 - binary_crossentropy: 0.31 - ETA: 10s - loss: 0.3216 - binary_